In [1]:
import h5py
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import os
import numpy as np
import glob

def compute_eer(far, frr):
    cords = zip(far, frr)
    min_dist = 999999
    for item in cords:
        item_far, item_frr = item
        dist = abs(item_far-item_frr)
        if dist < min_dist:
            min_dist = dist
            eer = (item_far + item_frr) / 2
    return eer

# Ped1 ground-truth
testSeqPath = '/usr/not-backed-up/1_DATABASE/UCSD_Anomaly_Dataset.tar/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/';
with h5py.File(os.path.join(testSeqPath,'TestFrameGT_original.mat')) as f:
    data = [f[element[0]][:] for element in f['FrameGt']]
data = np.array(data)
for seq in range(36):
    gt = data[seq]
    gt_ = gt[9:gt.shape[0],0]
    if seq == 0:
        groundtruth = gt_
    else:
        groundtruth = np.append(groundtruth,gt_)
# print(groundtruth.shape)        
# aug1
decision_path = '/usr/not-backed-up/1_convlstm/Ped1_prediction6/'

# aug2 
# decision_path = '/usr/not-backed-up/1_convlstm/Ped1_prediction_aug2/'
for i in range(36):
    print(i)
    filename = ("test_%d_error.h5" % (i+1))  
    h5f = h5py.File(os.path.join(decision_path,filename), 'r')
    frame_error = h5f['frame_error'][:]
    h5f.close()
    frame_error = frame_error - frame_error.min()
    frame_error = frame_error / frame_error.max()
    frame_error = np.squeeze(frame_error)
    if i == 0:
        error = frame_error
    else:
        error = np.append(error,frame_error)
#     print(error.shape)   
auc = roc_auc_score(groundtruth, error)

fpr, tpr, thresholds = roc_curve(groundtruth, error, pos_label=1)
frr = 1 - tpr
far = fpr
eer = compute_eer(far, frr)
print(eer, auc)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
(0.25157996749455547, 0.80807443012948221)


In [2]:
print(fpr, tpr, thresholds)

(array([ 0.        ,  0.00237208,  0.00237208, ...,  0.99051169,
        0.99051169,  1.        ]), array([ 0.        ,  0.00738854,  0.00764331, ...,  0.99770701,
        0.99796178,  1.        ]), array([  2.00000000e+00,   1.00000000e+00,   9.99132593e-01, ...,
         3.12129081e-04,   2.70812444e-04,   0.00000000e+00]))


In [4]:
print(len(thresholds))

2355
